In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.road_network import RoadNetwork

In [2]:
# load mapped dataset
df = pd.read_csv("../datasets/trajectories/Porto/train.csv")
# load network
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [3]:
# preprocess (clip trajectories to porto bounds and filter min points per trajectory (takes around 2h on pascal))
# trajectories that go out of porto and then again into porto area are splitted into seperated trajectories
fdf = preprocess_trajectories_porto(df, city_bounds=network.bounds, polyline_convert=True)
# save dataframe
fdf.to_csv("../datasets/trajectories/Porto/clipped_porto_full_10pmin_2mil.csv")

Pandas Apply:   0%|          | 0/1710660 [00:00<?, ?it/s]

/home/paul/repos/Road-Network-Embedding-Generator/generator/preprocess.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["POLYLINE"] = df["POLYLINE"].swifter.apply(convert_to_line_string)


Dask Apply:   0%|          | 0/24 [00:01<?, ?it/s]

In [4]:
"""
Map Timestamps to clipped dataframe. Each trajectory has a start timestamp and between each gps point are 15s.
We need to create a column which holds an array for each trajectory with the timestamps for each gps point.
"""
fdf = pd.read_csv("../datasets/trajectories/Porto/clipped_porto_full_10pmin_2mil.csv")



In [5]:
fdf.shape

(2152277, 11)

In [6]:
from ast import literal_eval
# Map trajectories in the format for fmm matching and insert timestamp column
df_fmm = fdf.loc[:, ["TRIP_ID", "POLYLINE", "coords"]]
df_fmm["coords"] = df_fmm["coords"].swifter.apply(literal_eval)
df_fmm["timestamp"] = df_fmm["coords"].swifter.apply(lambda x: list(np.arange(0, ((len(x)-1)*15)+1, 15)))
df_fmm["id"] = np.arange(1, df_fmm.shape[0]+1)
df_fmm = df_fmm.drop(["coords"], axis=1)
df_fmm.to_csv("../datasets/trajectories/Porto/mapped_id_poly_clipped_timestamp.csv", sep=";", index=False)

Pandas Apply:   0%|          | 0/2152277 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2152277 [00:00<?, ?it/s]

In [7]:
tdf = pd.read_csv("../datasets/trajectories/Porto/mapped_id_poly_clipped_timestamp.csv", sep=";")
tdf["timestamp"] = tdf["timestamp"].str.replace("[", "")
tdf["timestamp"] = tdf["timestamp"].str.replace("]", "")
tdf.to_csv("../datasets/trajectories/Porto/mapped_id_poly_clipped_timestamp.csv", sep=";", index=False)

In [25]:
from ast import literal_eval
# check timestamp length = gps stamp
tdf = pd.read_csv("../datasets/trajectories/Porto/mapped_id_poly_clipped_timestamp.csv", sep=";")
cdf= fdf.loc[:, ["coords"]]
cdf["coords"] = cdf["coords"].swifter.apply(literal_eval)
tdf["timestamp"] = tdf["timestamp"].swifter.apply(literal_eval)

In [20]:
df_fmm["timestamp"].str.len().mean()

29.01901335190591

In [26]:
tdf.shape

(2152277, 4)